In [65]:
import os
import pandas as pd
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4"

In [66]:
import torch
import random
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1)

In [67]:
df_weo = pd.read_excel('yearly/integrated_yearly_data.xlsx', index_col=0)
df_weo.index = [x for x in range(len(df_weo))]
# df_weo

In [68]:
df_weo = df_weo[[
       'Country Name', 'Country Code', 'year',
       'Rural population growth (annual %)',
       'General government final consumption expenditure (annual % growth)',
       'Consumer price index (2010 = 100)',
       'Exports of goods and services (annual % growth)_x',
       'Urban population growth (annual %)', 'GDP growth (annual %)',
       'Population growth (annual %)', 'Inflation, GDP deflator (annual %)',
       'Imports of goods and services (annual % growth)',
       'Final consumption expenditure (annual % growth)',
       'Unemployment, total (% of total labor force) (national estimate)',
       'Inflation, consumer prices (annual %)',
       'Gross fixed capital formation (annual % growth)',
       'Households and NPISHs Final consumption expenditure (annual % growth)'
]]

In [69]:
df_weo = df_weo.dropna(subset=['GDP growth (annual %)'])
df_weo.index = [x for x in range(len(df_weo))]

In [70]:
df_weo['year'].unique()

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [72]:
df_weo = df_weo.loc[(df_weo['year'] >= 2013) & (df_weo['year'] <= 2019)]

In [73]:
df_merge = df_weo.dropna()
df_merge

,Country Name,Country Code,year,Rural population growth (annual %),General government final consumption expenditure (annual % growth),Consumer price index (2010 = 100),Exports of goods and services (annual % growth)_x,Urban population growth (annual %),GDP growth (annual %),Population growth (annual %),"Inflation, GDP deflator (annual %)",Imports of goods and services (annual % growth),Final consumption expenditure (annual % growth),"Unemployment, total (% of total labor force) (national estimate)","Inflation, consumer prices (annual %)",Gross fixed capital formation (annual % growth),Households and NPISHs Final consumption expenditure (annual % growth)
540,Australia,AUS,2013,1.033771,0.571659,107.700312,5.305435,1.838176,2.606198,1.721151,-0.163782,0.697428,1.564854,5.663,2.449889,2.891657,1.915095
541,Brazil,BRA,2013,-1.046804,1.510121,119.372104,1.830295,1.204556,3.004823,0.868346,7.504565,6.669679,3.019444,7.071,6.204311,5.827201,3.474859
542,Canada,CAN,2013,0.838999,-0.789646,105.452204,2.453414,1.107110,2.325814,1.056591,1.740738,2.081262,1.686292,7.143,0.938292,1.448880,2.627830
543,Switzerland,CHE,2013,1.070130,1.799332,99.320878,14.171961,1.178392,1.792144,1.149880,-0.086843,12.268706,2.178382,4.747,-0.217323,1.118239,2.259422
544,Germany,DEU,2013,0.229090,1.411439,105.692140,1.004464,0.285857,0.437591,0.272900,1.959392,2.675547,0.635020,5.231,1.504723,-1.296008,0.361832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,Poland,POL,2019,0.028134,6.482303,114.111779,5.337289,-0.059399,4.449988,-0.024427,3.032537,3.192354,4.114586,3.277,2.227479,6.242051,3.404616
671,Russian Federation,RUS,2019,-0.653729,2.369754,180.750264,0.732545,0.157115,2.198076,-0.049569,3.263133,3.130358,3.389621,4.496,4.470367,0.966743,3.753272
672,Saudi Arabia,SAU,2019,0.907224,-2.971078,118.399825,-4.996865,2.547829,0.832280,2.284589,-1.764821,9.580893,2.968098,5.636,-2.093333,4.827888,6.488206
673,Turkiye,TUR,2019,-0.548956,3.907612,234.437126,5.278520,2.075703,0.818526,1.429697,13.867735,-5.011971,1.965464,13.730,15.176822,-12.471792,1.504079


## add light

In [77]:
df_code = pd.read_excel('IMF_ISO.xlsx', header=None)
df_code.columns = ['REF_AREA', 'ADM0_ISO']
# df_code

In [78]:
df_light_data = pd.read_csv('data/light_data.csv', index_col=0)
df_light_data = df_light_data.merge(df_code, how='left', on='ADM0_ISO')
df_light_data = df_light_data.dropna(subset=['REF_AREA'])
df_light_data['REF_AREA'] = df_light_data['REF_AREA'].astype(int)
# df_light_data

In [79]:
df_light_data = df_light_data.loc[(df_light_data['year_month'] >= 201301) & \
                                  (df_light_data['year_month'] <= 202412)]

In [80]:
# drop some country nan value
df_light_data = pd.DataFrame(df_light_data.groupby(['ADM0_ISO', 'year_month', 'REF_AREA']).max('sum').reset_index())

In [81]:
# 将字符串转换为日期时间格式
df_light_data['year_month'] = pd.to_datetime(df_light_data['year_month'], format='%Y%m')

# 提取季度信息
df_light_data['year'] = df_light_data['year_month'].dt.year
df_light_data['month'] = df_light_data['year_month'].dt.month

In [82]:
df_light_data

,ADM0_ISO,year_month,REF_AREA,sum,mean,std,year,month
0,ABW,2013-01-01,314,3236.639999,4.035711,6.253305,2013,1
1,ABW,2013-02-01,314,3422.669997,4.235978,5.449601,2013,2
2,ABW,2013-03-01,314,3059.299997,3.786262,4.239345,2013,3
3,ABW,2013-04-01,314,3186.650002,3.943874,4.771718,2013,4
4,ABW,2013-05-01,314,3253.700000,4.026856,5.988211,2013,5
...,...,...,...,...,...,...,...,...
25398,ZWE,2023-09-01,698,781236.070851,0.255524,1.733717,2023,9
25399,ZWE,2023-10-01,698,934729.121041,0.305728,2.026455,2023,10
25400,ZWE,2023-11-01,698,958243.855843,0.313420,0.937193,2023,11
25401,ZWE,2023-12-01,698,661726.530320,0.216734,1.098943,2023,12


In [83]:
# average of year
df_light_data = df_light_data.groupby(['ADM0_ISO', 'REF_AREA', 'year']).mean().reset_index()
df_light_data

,ADM0_ISO,REF_AREA,year,year_month,sum,mean,std,month
0,ABW,314,2013,2013-06-16 12:00:00,3285.883333,4.069567,5.464744,6.5
1,ABW,314,2014,2014-06-16 12:00:00,3542.806667,4.398952,5.539452,6.5
2,ABW,314,2015,2015-06-16 12:00:00,3616.676666,4.478046,5.527158,6.5
3,ABW,314,2016,2016-06-16 08:00:00,3965.599998,4.916097,6.062411,6.5
4,ABW,314,2017,2017-06-16 12:00:00,5051.679166,6.252078,7.150565,6.5
...,...,...,...,...,...,...,...,...
2287,ZWE,698,2020,2020-06-16 08:00:00,483571.902194,0.251580,0.564372,6.5
2288,ZWE,698,2021,2021-06-16 12:00:00,547103.053408,0.178995,0.626771,6.5
2289,ZWE,698,2022,2022-06-16 12:00:00,571825.590048,0.187945,0.663419,6.5
2290,ZWE,698,2023,2023-06-16 12:00:00,666154.859491,0.217988,1.065642,6.5


In [84]:
df_merge['country'] = df_merge['Country Code']

In [85]:
dict_country_code = {}
for i in range(len(df_merge['country'].unique())):
    dict_country_code[df_merge['country'].unique()[i]] = ['CHN', 'USA', 'DEU', 'JPN', 'IND', 'GBR', 'FRA', 'BRA', 'ITA', 'CAN', 'RUS', 'MEX',
 'AUS', 'KOR', 'ESP', 'IDN', 'NLD', 'TUR', 'SAU', 'CHE', 'POL'][i]
dict_country_code = dict((v, k) for k, v in dict_country_code.items())
df_light_data_select = df_light_data.loc[df_light_data['ADM0_ISO'].isin(dict_country_code.keys())][['ADM0_ISO', 'year', 'sum', 'mean', 'std']]

df_light_data_select['country'] = df_light_data_select['ADM0_ISO'].apply(lambda x: dict_country_code[x])
df_light_data_select

,ADM0_ISO,year,sum,mean,std,country
108,AUS,2013,4.025313e+06,0.156927,6.314056,MEX
109,AUS,2014,3.797938e+06,0.409307,7.559085,MEX
110,AUS,2015,4.048216e+06,0.521269,9.966388,MEX
111,AUS,2016,2.577938e+06,0.329611,6.365509,MEX
112,AUS,2017,8.987109e+06,0.248867,4.003674,MEX
...,...,...,...,...,...,...
2167,USA,2020,5.768624e+07,0.897867,6.613326,BRA
2168,USA,2021,6.156864e+07,0.056712,1.596962,BRA
2169,USA,2022,7.049229e+07,0.064932,1.750268,BRA
2170,USA,2023,6.794478e+07,0.062586,1.681313,BRA


In [86]:
df_merge = pd.merge(df_merge, df_light_data_select, how='left', on=['country', 'year'])

In [87]:
df_merge

,Country Name,Country Code,year,Rural population growth (annual %),General government final consumption expenditure (annual % growth),Consumer price index (2010 = 100),Exports of goods and services (annual % growth)_x,Urban population growth (annual %),GDP growth (annual %),Population growth (annual %),...,Final consumption expenditure (annual % growth),"Unemployment, total (% of total labor force) (national estimate)","Inflation, consumer prices (annual %)",Gross fixed capital formation (annual % growth),Households and NPISHs Final consumption expenditure (annual % growth),country,ADM0_ISO,sum,mean,std
0,Australia,AUS,2013,1.033771,0.571659,107.700312,5.305435,1.838176,2.606198,1.721151,...,1.564854,5.663,2.449889,2.891657,1.915095,AUS,CHN,2.173635e+07,0.408269,3.095361
1,Brazil,BRA,2013,-1.046804,1.510121,119.372104,1.830295,1.204556,3.004823,0.868346,...,3.019444,7.071,6.204311,5.827201,3.474859,BRA,USA,5.387293e+07,0.847616,5.971115
2,Canada,CAN,2013,0.838999,-0.789646,105.452204,2.453414,1.107110,2.325814,1.056591,...,1.686292,7.143,0.938292,1.448880,2.627830,CAN,DEU,2.048263e+06,0.777476,2.729972
3,Switzerland,CHE,2013,1.070130,1.799332,99.320878,14.171961,1.178392,1.792144,1.149880,...,2.178382,4.747,-0.217323,1.118239,2.259422,CHE,JPN,2.989069e+06,1.366240,5.156093
4,Germany,DEU,2013,0.229090,1.411439,105.692140,1.004464,0.285857,0.437591,0.272900,...,0.635020,5.231,1.504723,-1.296008,0.361832,DEU,IND,9.329172e+06,0.604302,3.175042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Poland,POL,2019,0.028134,6.482303,114.111779,5.337289,-0.059399,4.449988,-0.024427,...,4.114586,3.277,2.227479,6.242051,3.404616,POL,ESP,4.078532e+06,1.319188,6.476156
131,Russian Federation,RUS,2019,-0.653729,2.369754,180.750264,0.732545,0.157115,2.198076,-0.049569,...,3.389621,4.496,4.470367,0.966743,3.753272,RUS,IDN,3.502264e+06,0.412781,3.096644
132,Saudi Arabia,SAU,2019,0.907224,-2.971078,118.399825,-4.996865,2.547829,0.832280,2.284589,...,2.968098,5.636,-2.093333,4.827888,6.488206,SAU,NLD,1.214743e+06,4.296103,50.873116
133,Turkiye,TUR,2019,-0.548956,3.907612,234.437126,5.278520,2.075703,0.818526,1.429697,...,1.965464,13.730,15.176822,-12.471792,1.504079,TUR,TUR,5.668421e+06,1.212351,5.090605


In [88]:
df_filter = df_merge.dropna()

In [ ]:
data = df_filter[data_v]

In [ ]:
data.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data.values)

In [ ]:
df_filter[data_v] = data

In [ ]:
df_filter.head()

## Save

In [89]:
country_list = df_filter['Country Name'].unique()

In [90]:
code_map = {}
for i in range(len(country_list)):
    code_map[country_list[i]] = i

In [91]:
df_filter['country_code'] = df_filter['Country Name'].apply(lambda x: code_map[x])

In [92]:
df_filter

,Country Name,Country Code,year,Rural population growth (annual %),General government final consumption expenditure (annual % growth),Consumer price index (2010 = 100),Exports of goods and services (annual % growth)_x,Urban population growth (annual %),GDP growth (annual %),Population growth (annual %),...,"Unemployment, total (% of total labor force) (national estimate)","Inflation, consumer prices (annual %)",Gross fixed capital formation (annual % growth),Households and NPISHs Final consumption expenditure (annual % growth),country,ADM0_ISO,sum,mean,std,country_code
0,Australia,AUS,2013,1.033771,0.571659,107.700312,5.305435,1.838176,2.606198,1.721151,...,5.663,2.449889,2.891657,1.915095,AUS,CHN,2.173635e+07,0.408269,3.095361,0
1,Brazil,BRA,2013,-1.046804,1.510121,119.372104,1.830295,1.204556,3.004823,0.868346,...,7.071,6.204311,5.827201,3.474859,BRA,USA,5.387293e+07,0.847616,5.971115,1
2,Canada,CAN,2013,0.838999,-0.789646,105.452204,2.453414,1.107110,2.325814,1.056591,...,7.143,0.938292,1.448880,2.627830,CAN,DEU,2.048263e+06,0.777476,2.729972,2
3,Switzerland,CHE,2013,1.070130,1.799332,99.320878,14.171961,1.178392,1.792144,1.149880,...,4.747,-0.217323,1.118239,2.259422,CHE,JPN,2.989069e+06,1.366240,5.156093,3
4,Germany,DEU,2013,0.229090,1.411439,105.692140,1.004464,0.285857,0.437591,0.272900,...,5.231,1.504723,-1.296008,0.361832,DEU,IND,9.329172e+06,0.604302,3.175042,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Poland,POL,2019,0.028134,6.482303,114.111779,5.337289,-0.059399,4.449988,-0.024427,...,3.277,2.227479,6.242051,3.404616,POL,ESP,4.078532e+06,1.319188,6.476156,14
131,Russian Federation,RUS,2019,-0.653729,2.369754,180.750264,0.732545,0.157115,2.198076,-0.049569,...,4.496,4.470367,0.966743,3.753272,RUS,IDN,3.502264e+06,0.412781,3.096644,15
132,Saudi Arabia,SAU,2019,0.907224,-2.971078,118.399825,-4.996865,2.547829,0.832280,2.284589,...,5.636,-2.093333,4.827888,6.488206,SAU,NLD,1.214743e+06,4.296103,50.873116,16
133,Turkiye,TUR,2019,-0.548956,3.907612,234.437126,5.278520,2.075703,0.818526,1.429697,...,13.730,15.176822,-12.471792,1.504079,TUR,TUR,5.668421e+06,1.212351,5.090605,17


In [93]:
data_v = [
       'Rural population growth (annual %)',
       'General government final consumption expenditure (annual % growth)',
       'Consumer price index (2010 = 100)',
       'Exports of goods and services (annual % growth)_x',
       'Urban population growth (annual %)',
       'Population growth (annual %)',
       'Inflation, GDP deflator (annual %)',
       'Imports of goods and services (annual % growth)',
       'Final consumption expenditure (annual % growth)',
       'Unemployment, total (% of total labor force) (national estimate)',
       'Inflation, consumer prices (annual %)',
       'Gross fixed capital formation (annual % growth)',
       'Households and NPISHs Final consumption expenditure (annual % growth)',
       'sum', 'mean', 'std',
       'country_code', 'year'
]

In [94]:
len(data_v)

16

In [95]:
label_v = ['GDP growth (annual %)',
          'country_code', 'year']

In [1]:
data = torch.Tensor(df_filter[data_v].to_numpy())
data.size()

In [2]:
labels = torch.Tensor(df_filter[label_v].to_numpy())
labels.size()

In [98]:
labels[0]

tensor([   2.6062,    0.0000, 2013.0000])

In [99]:
torch.save(data, 'dataset/MLP_data_light_sms_y_13-19.pt')

In [100]:
torch.save(labels, 'dataset/MLP_label_light_sms_y_13-19.pt')